<a href="https://colab.research.google.com/github/isshii/de4test/blob/master/DeepXplore/DeepXplore_with_LimeData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# 共通で使うパスなどの定義
# 共通の変数設定
# 共通フォルダパス
data_dir = "/content/gdrive/My Drive/ColabNotebooks/test4ai/data"
data_imagenet = "/content/gdrive/My Drive/ColabNotebooks/test4ai/data/ImageNet"
data_imagenet_seeds = "/content/gdrive/My Drive/ColabNotebooks/test4ai/data/ImageNet/seeds"
model_dir = "/content/gdrive/My Drive/ColabNotebooks/test4ai/model"
output_dir = "/content/gdrive/My Drive/ColabNotebooks/test4ai/output"
tmp_dir = "/content/gdrive/My Drive/ColabNotebooks/test4ai/tmp"


In [4]:
# Goggle Drive つなぐ
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# outputフォルダ内容物のクリーンアップ削除
!rm "$output_dir"/*

In [5]:
import argparse

%tensorflow_version 1.x
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.layers import Input
import imageio


Using TensorFlow backend.


In [6]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 473040390303178362, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 17029367125077635141
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 9661749827306392005
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14912199066
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 16806230322597559401
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [0]:
import random
from collections import defaultdict

import numpy as np
from keras import backend as K
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.models import Model
from keras.preprocessing import image
import os, re


def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    input_img_data = image.img_to_array(img)
    input_img_data = np.expand_dims(input_img_data, axis=0)
    input_img_data = preprocess_input(input_img_data)  # final input shape = (1,224,224,3)
    return input_img_data


def deprocess_image(x):
    x = x.reshape((224, 224, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x


def decode_label(pred):
    return decode_predictions(pred)[0][0][1]


def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + 1e-5)


def constraint_occl(gradients, start_point, rect_shape):
    new_grads = np.zeros_like(gradients)
    new_grads[:, start_point[0]:start_point[0] + rect_shape[0],
    start_point[1]:start_point[1] + rect_shape[1]] = gradients[:, start_point[0]:start_point[0] + rect_shape[0],
                                                     start_point[1]:start_point[1] + rect_shape[1]]
    return new_grads


def constraint_light(gradients):
    new_grads = np.ones_like(gradients)
    grad_mean = 1e4 * np.mean(gradients)
    return grad_mean * new_grads


def constraint_black(gradients, rect_shape=(10, 10)):
    start_point = (
        random.randint(0, gradients.shape[1] - rect_shape[0]), random.randint(0, gradients.shape[2] - rect_shape[1]))
    new_grads = np.zeros_like(gradients)
    patch = gradients[:, start_point[0]:start_point[0] + rect_shape[0], start_point[1]:start_point[1] + rect_shape[1]]
    if np.mean(patch) < 0:
        new_grads[:, start_point[0]:start_point[0] + rect_shape[0],
        start_point[1]:start_point[1] + rect_shape[1]] = -np.ones_like(patch)
    return new_grads


def init_coverage_tables(model1, model2, model3):
    model_layer_dict1 = defaultdict(bool)
    model_layer_dict2 = defaultdict(bool)
    model_layer_dict3 = defaultdict(bool)
    init_dict(model1, model_layer_dict1)
    init_dict(model2, model_layer_dict2)
    init_dict(model3, model_layer_dict3)
    return model_layer_dict1, model_layer_dict2, model_layer_dict3


def init_dict(model, model_layer_dict):
    for layer in model.layers:
        if 'flatten' in layer.name or 'input' in layer.name:
            continue
        for index in range(layer.output_shape[-1]):
            model_layer_dict[(layer.name, index)] = False


def neuron_to_cover(model_layer_dict):
    not_covered = [(layer_name, index) for (layer_name, index), v in list(model_layer_dict.items()) if not v]
    if not_covered:
        layer_name, index = random.choice(not_covered)
    else:
        layer_name, index = random.choice(list(model_layer_dict.keys()))
    return layer_name, index


def neuron_covered(model_layer_dict):
    covered_neurons = len([v for v in list(model_layer_dict.values()) if v])
    total_neurons = len(model_layer_dict)
    return covered_neurons, total_neurons, covered_neurons / float(total_neurons)


def scale(intermediate_layer_output, rmax=1, rmin=0):
    X_std = (intermediate_layer_output - intermediate_layer_output.min()) / (
        intermediate_layer_output.max() - intermediate_layer_output.min())
    X_scaled = X_std * (rmax - rmin) + rmin
    return X_scaled


def update_coverage(input_data, model, model_layer_dict, threshold=0):
    layer_names = [layer.name for layer in model.layers if
                   'flatten' not in layer.name and 'input' not in layer.name]

    intermediate_layer_model = Model(inputs=model.input,
                                     outputs=[model.get_layer(layer_name).output for layer_name in layer_names])
    intermediate_layer_outputs = intermediate_layer_model.predict(input_data)

    for i, intermediate_layer_output in enumerate(intermediate_layer_outputs):
        scaled = scale(intermediate_layer_output[0])
        for num_neuron in range(scaled.shape[-1]):
            if np.mean(scaled[..., num_neuron]) > threshold and not model_layer_dict[(layer_names[i], num_neuron)]:
                model_layer_dict[(layer_names[i], num_neuron)] = True


def full_coverage(model_layer_dict):
    if False in list(model_layer_dict.values()):
        return False
    return True


def fired(model, layer_name, index, input_data, threshold=0):
    intermediate_layer_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
    intermediate_layer_output = intermediate_layer_model.predict(input_data)[0]
    scaled = scale(intermediate_layer_output)
    if np.mean(scaled[..., index]) > threshold:
        return True
    return False


def diverged(predictions1, predictions2, predictions3, target):
    if not predictions1 == predictions2 == predictions3:
        return True
    return False

#add
def list_pictures(directory, ext='jpg|jpeg|bmp|png|ppm'):
    return [os.path.join(root, f)
            for root, _, files in os.walk(directory) for f in files
            if re.match(r'([\w]+\.(?:' + ext + '))', f.lower())]

In [8]:
%%time
# input image dimensions
img_rows, img_cols = 224, 224
input_shape = (img_rows, img_cols, 3)

# define input tensor as a placeholder
input_tensor = Input(shape=input_shape)

# load multiple models sharing same input tensor
K.set_learning_phase(0)
model1 = VGG16(input_tensor=input_tensor)
model2 = VGG19(input_tensor=input_tensor)
model3 = ResNet50(input_tensor=input_tensor)


# init coverage table
model_layer_dict1, model_layer_dict2, model_layer_dict3 = init_coverage_tables(model1, model2, model3)





553467904/553467096 [==============================] - 6s 0us/step






574717952/574710816 [==============================] - 19s 0us/step


102858752/102853048 [==============================] - 1s 0us/step
CPU times: user 11 s, sys: 5.13 s, total: 16.2 s
Wall time: 42.2 s


In [9]:
import yaml
class PrettySafeLoader(yaml.SafeLoader):
    def construct_python_tuple(self, node):
        return tuple(self.construct_sequence(node))

PrettySafeLoader.add_constructor(
    u'tag:yaml.org,2002:python/tuple',
    PrettySafeLoader.construct_python_tuple)

thing = yaml.load("""
ILSVRC2012_test_00001288.JPEG: !!python/tuple [164, 0]
ILSVRC2012_test_00001306.JPEG: !!python/tuple [62, 109]
ILSVRC2012_test_00001322.JPEG: !!python/tuple [0, 36]
ILSVRC2012_test_00001351.JPEG: !!python/tuple [118, 5]
ILSVRC2012_test_00001369.JPEG: !!python/tuple [119, 119]
ILSVRC2012_test_00001375.JPEG: !!python/tuple [54, 70]
ILSVRC2012_test_00001390.JPEG: !!python/tuple [123, 0]
ILSVRC2012_test_00001422.JPEG: !!python/tuple [137, 118]
ILSVRC2012_test_00001428.JPEG: !!python/tuple [16, 73]
ILSVRC2012_test_00001451.JPEG: !!python/tuple [141, 12]
ILSVRC2012_test_00001469.JPEG: !!python/tuple [44, 164]
ILSVRC2012_test_00001475.JPEG: !!python/tuple [70, 164]
ILSVRC2012_test_00001490.JPEG: !!python/tuple [97, 0]
ILSVRC2012_test_00001514.JPEG: !!python/tuple [164, 145]
ILSVRC2012_test_00001526.JPEG: !!python/tuple [140, 20]
ILSVRC2012_test_00001530.JPEG: !!python/tuple [164, 164]
ILSVRC2012_test_00001543.JPEG: !!python/tuple [72, 90]
ILSVRC2012_test_00001549.JPEG: !!python/tuple [104, 55]
ILSVRC2012_test_00001567.JPEG: !!python/tuple [77, 122]
ILSVRC2012_test_00001582.JPEG: !!python/tuple [60, 70]
ILSVRC2012_test_00001588.JPEG: !!python/tuple [164, 164]
ILSVRC2012_test_00001613.JPEG: !!python/tuple [0, 85]
ILSVRC2012_test_00001619.JPEG: !!python/tuple [47, 25]
ILSVRC2012_test_00001637.JPEG: !!python/tuple [0, 135]
ILSVRC2012_test_00001644.JPEG: !!python/tuple [29, 156]
ILSVRC2012_test_00001660.JPEG: !!python/tuple [164, 0]
ILSVRC2012_test_00001676.JPEG: !!python/tuple [0, 57]
ILSVRC2012_test_00001685.JPEG: !!python/tuple [121, 23]
ILSVRC2012_test_00001699.JPEG: !!python/tuple [164, 23]
ILSVRC2012_test_00001701.JPEG: !!python/tuple [80, 164]
ILSVRC2012_test_00001725.JPEG: !!python/tuple [56, 164]
ILSVRC2012_test_00001733.JPEG: !!python/tuple [71, 43]
ILSVRC2012_test_00001739.JPEG: !!python/tuple [0, 164]
ILSVRC2012_test_00001756.JPEG: !!python/tuple [0, 164]
ILSVRC2012_test_00001772.JPEG: !!python/tuple [88, 87]
ILSVRC2012_test_00001778.JPEG: !!python/tuple [29, 164]
ILSVRC2012_test_00001796.JPEG: !!python/tuple [118, 164]
ILSVRC2012_test_00001797.JPEG: !!python/tuple [164, 164]
ILSVRC2012_test_00001810.JPEG: !!python/tuple [136, 86]
ILSVRC2012_test_00001834.JPEG: !!python/tuple [16, 68]
ILSVRC2012_test_00001847.JPEG: !!python/tuple [99, 54]
ILSVRC2012_test_00001863.JPEG: !!python/tuple [123, 4]
ILSVRC2012_test_00001869.JPEG: !!python/tuple [67, 137]
ILSVRC2012_test_00001886.JPEG: !!python/tuple [141, 75]
ILSVRC2012_test_00001890.JPEG: !!python/tuple [164, 164]
ILSVRC2012_test_00001902.JPEG: !!python/tuple [104, 43]
ILSVRC2012_test_00001926.JPEG: !!python/tuple [62, 135]
ILSVRC2012_test_00001930.JPEG: !!python/tuple [21, 164]
ILSVRC2012_test_00001949.JPEG: !!python/tuple [117, 17]
ILSVRC2012_test_00001955.JPEG: !!python/tuple [0, 80]
""")

for filename in thing:
    print(f"{filename} : {thing[filename]}")
    # ここで dx
    # filename は使い方同じで、thing[filename] をノイズの場所として与える
    

ILSVRC2012_test_00001288.JPEG : (164, 0)
ILSVRC2012_test_00001306.JPEG : (62, 109)
ILSVRC2012_test_00001322.JPEG : (0, 36)
ILSVRC2012_test_00001351.JPEG : (118, 5)
ILSVRC2012_test_00001369.JPEG : (119, 119)
ILSVRC2012_test_00001375.JPEG : (54, 70)
ILSVRC2012_test_00001390.JPEG : (123, 0)
ILSVRC2012_test_00001422.JPEG : (137, 118)
ILSVRC2012_test_00001428.JPEG : (16, 73)
ILSVRC2012_test_00001451.JPEG : (141, 12)
ILSVRC2012_test_00001469.JPEG : (44, 164)
ILSVRC2012_test_00001475.JPEG : (70, 164)
ILSVRC2012_test_00001490.JPEG : (97, 0)
ILSVRC2012_test_00001514.JPEG : (164, 145)
ILSVRC2012_test_00001526.JPEG : (140, 20)
ILSVRC2012_test_00001530.JPEG : (164, 164)
ILSVRC2012_test_00001543.JPEG : (72, 90)
ILSVRC2012_test_00001549.JPEG : (104, 55)
ILSVRC2012_test_00001567.JPEG : (77, 122)
ILSVRC2012_test_00001582.JPEG : (60, 70)
ILSVRC2012_test_00001588.JPEG : (164, 164)
ILSVRC2012_test_00001613.JPEG : (0, 85)
ILSVRC2012_test_00001619.JPEG : (47, 25)
ILSVRC2012_test_00001637.JPEG : (0, 135)
IL

In [0]:
def deep_xplorer(img_path):
    gen_img = preprocess_image(img_path)
    orig_img = gen_img.copy()
    # first check if input already induces differences
    pred1, pred2, pred3 = model1.predict(gen_img), model2.predict(gen_img), model3.predict(gen_img)
    label1, label2, label3 = np.argmax(pred1[0]), np.argmax(pred2[0]), np.argmax(pred3[0])
    if not label1 == label2 == label3:
        print('input already causes different outputs: {}, {}, {}'.format(decode_label(pred1),
                                                                                            decode_label(pred2),
                                                                                            decode_label(
                                                                                                pred3)))

        update_coverage(gen_img, model1, model_layer_dict1, args.threshold)
        update_coverage(gen_img, model2, model_layer_dict2, args.threshold)
        update_coverage(gen_img, model3, model_layer_dict3, args.threshold)

        print('covered neurons percentage %d neurons %.3f, %d neurons %.3f, %d neurons %.3f'
              % (len(model_layer_dict1), neuron_covered(model_layer_dict1)[2], len(model_layer_dict2),
                 neuron_covered(model_layer_dict2)[2], len(model_layer_dict3),
                 neuron_covered(model_layer_dict3)[2]))
  
        averaged_nc = (neuron_covered(model_layer_dict1)[0] + neuron_covered(model_layer_dict2)[0] +
                       neuron_covered(model_layer_dict3)[0]) / float(
            neuron_covered(model_layer_dict1)[1] + neuron_covered(model_layer_dict2)[1] +
            neuron_covered(model_layer_dict3)[1])
        print('averaged covered neurons %.3f' % averaged_nc)

        gen_img_deprocessed = deprocess_image(gen_img)

        # save the result to disk
        outputfilepath0 = os.path.join(output_dir, 'already_differ_' + decode_label(pred1) + '_' + decode_label(pred2) + '_' + decode_label(pred3) + '.png')
        imageio.imwrite(outputfilepath0, gen_img_deprocessed)
        return

    # if all label agrees
    orig_label = label1
    layer_name1, index1 = neuron_to_cover(model_layer_dict1)
    layer_name2, index2 = neuron_to_cover(model_layer_dict2)
    layer_name3, index3 = neuron_to_cover(model_layer_dict3)

    # construct joint loss function
    if args.target_model == 0:
        loss1 = -args.weight_diff * K.mean(model1.get_layer('predictions').output[..., orig_label])
        loss2 = K.mean(model2.get_layer('predictions').output[..., orig_label])
        loss3 = K.mean(model3.get_layer('fc1000').output[..., orig_label])
    elif args.target_model == 1:
        loss1 = K.mean(model1.get_layer('predictions').output[..., orig_label])
        loss2 = -args.weight_diff * K.mean(model2.get_layer('predictions').output[..., orig_label])
        loss3 = K.mean(model3.get_layer('fc1000').output[..., orig_label])
    elif args.target_model == 2:
        loss1 = K.mean(model1.get_layer('predictions').output[..., label1])
        loss2 = K.mean(model2.get_layer('predictions').output[..., orig_label])
        loss3 = -args.weight_diff * K.mean(model3.get_layer('fc1000').output[..., orig_label])
    loss1_neuron = K.mean(model1.get_layer(layer_name1).output[..., index1])
    loss2_neuron = K.mean(model2.get_layer(layer_name2).output[..., index2])
    loss3_neuron = K.mean(model3.get_layer(layer_name3).output[..., index3])
    layer_output = (loss1 + loss2 + loss3) + args.weight_nc * (loss1_neuron + loss2_neuron + loss3_neuron)

    # for adversarial image generation
    final_loss = K.mean(layer_output)

    # we compute the gradient of the input picture wrt this loss
    grads = normalize(K.gradients(final_loss, input_tensor)[0])

    # this function returns the loss and grads given the input picture
    iterate = K.function([input_tensor], [loss1, loss2, loss3, loss1_neuron, loss2_neuron, loss3_neuron, grads])

    # we run gradient ascent for 20 steps
    for iters in range(args.grad_iterations):
        loss_value1, loss_value2, loss_value3, loss_neuron1, loss_neuron2, loss_neuron3, grads_value = iterate(
            [gen_img])
        if args.transformation == 'light':
            grads_value = constraint_light(grads_value)  # constraint the gradients value
        elif args.transformation == 'occl':
            grads_value = constraint_occl(grads_value, args.start_point,
                                          args.occlusion_size)  # constraint the gradients value
        elif args.transformation == 'blackout':
            grads_value = constraint_black(grads_value)  # constraint the gradients value

        gen_img += grads_value * args.step
        pred1, pred2, pred3 = model1.predict(gen_img), model2.predict(gen_img), model3.predict(gen_img)
        label1, label2, label3 = np.argmax(pred1[0]), np.argmax(pred2[0]), np.argmax(pred3[0])

        if not label1 == label2 == label3:
            update_coverage(gen_img, model1, model_layer_dict1, args.threshold)
            update_coverage(gen_img, model2, model_layer_dict2, args.threshold)
            update_coverage(gen_img, model3, model_layer_dict3, args.threshold)
            averaged_nc = (neuron_covered(model_layer_dict1)[0] + neuron_covered(model_layer_dict2)[0] +
                           neuron_covered(model_layer_dict3)[0]) / float(
                neuron_covered(model_layer_dict1)[1] + neuron_covered(model_layer_dict2)[1] +
                neuron_covered(model_layer_dict3)[
                    1])
            print('averaged covered neurons %.3f' % averaged_nc)

            gen_img_deprocessed = deprocess_image(gen_img)
            orig_img_deprocessed = deprocess_image(orig_img)

            # save the result to disk
            outputfilepath = os.path.join(output_dir, args.transformation + '_' + decode_label(pred1) + '_' + decode_label(pred2) + '_' + decode_label(pred3) + '.png')
            print(outputfilepath)
            imageio.imwrite(outputfilepath, gen_img_deprocessed)
            outputfilepath2 = os.path.join(output_dir, args.transformation + '_' + decode_label(pred1) + '_' + decode_label(pred2) + '_' + decode_label(pred3) + '_orig.png')
            print(outputfilepath2)
            imageio.imwrite(outputfilepath2, orig_img_deprocessed)
            break

In [0]:
# start gen inputs
 img_paths = list_pictures(data_imagenet_seeds)

In [0]:
%%time
iteration = 1
for img_path in img_paths:
  print(iteration)
  iteration += 1
  print(os.path.basename(img_path))
  print(thing[os.path.basename(img_path)])
  start_x = 0 # thing[os.path.basename(img_path)][0]
  start_y = 0 # thing[os.path.basename(img_path)][1]


  parser = argparse.ArgumentParser(description='Main function for difference-inducing input generation in MNIST dataset')
  parser.add_argument('transformation', help="realistic transformation type", choices=['light', 'occl', 'blackout'])
  parser.add_argument('weight_diff', help="weight hyperparm to control differential behavior", type=float)
  parser.add_argument('weight_nc', help="weight hyperparm to control neuron coverage", type=float)
  parser.add_argument('step', help="step size of gradient descent", type=float)
  parser.add_argument('seeds', help="number of seeds of input", type=int)
  parser.add_argument('grad_iterations', help="number of iterations of gradient descent", type=int)
  parser.add_argument('threshold', help="threshold for determining neuron activated", type=float)
  parser.add_argument('-t', '--target_model', help="target model that we want it predicts differently",
                    choices=[0, 1, 2], default=0, type=int)
  parser.add_argument('-sp', '--start_point', help="occlusion upper left corner coordinate", default=(start_y, start_x), type=tuple)
  parser.add_argument('-occl_size', '--occlusion_size', help="occlusion size", default=(60, 60), type=tuple)

  noise_type = "occl" 
  weight_diff = "0.1" 
  weight_nc = "0.1" 
  step = "1" 
  seeds = "50" 
  grad_iterations = "100" 
  threshold = "0.1"

  args = parser.parse_args([noise_type, weight_diff, weight_nc, step, seeds, grad_iterations, threshold])
  
  deep_xplorer(img_path)

1
ILSVRC2012_test_00001926.JPEG
(62, 135)
input already causes different outputs: stole, stole, cardigan
covered neurons percentage 14888 neurons 0.607, 16168 neurons 0.584, 94123 neurons 0.827
averaged covered neurons 0.769
2
ILSVRC2012_test_00001869.JPEG
(67, 137)
input already causes different outputs: bannister, bannister, pole
covered neurons percentage 14888 neurons 0.607, 16168 neurons 0.584, 94123 neurons 0.827
averaged covered neurons 0.769
3
ILSVRC2012_test_00001733.JPEG
(71, 43)
4
ILSVRC2012_test_00001567.JPEG
(77, 122)
5
ILSVRC2012_test_00001739.JPEG
(0, 164)
6
ILSVRC2012_test_00001810.JPEG
(136, 86)
7
ILSVRC2012_test_00001288.JPEG
(164, 0)
input already causes different outputs: convertible, sports_car, convertible
covered neurons percentage 14888 neurons 0.607, 16168 neurons 0.584, 94123 neurons 0.827
averaged covered neurons 0.769
8
ILSVRC2012_test_00001514.JPEG
(164, 145)
9
ILSVRC2012_test_00001490.JPEG
(97, 0)
10
ILSVRC2012_test_00001902.JPEG
(104, 43)
averaged covered